Keywords:
- TODO: things missing or to be checked
- ASSUMPTIONS: any assumptions taken on the implementation

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pandapower.plotting.plotly import simple_plotly

In [ ]:
import donotcommit
ean_coor, \
    armoire_resa, coupe_liste_resa, lg_rappel, noms_voiries, postes_resa, supports_resa, troncon_resa, voirie_bord, \
    under_lines, under_points, under_polys, \
sub_tfo, \
    m105, \
    bi_assets = donotcommit.ImportPaths()

gis_ean_coor = pd.read_csv(ean_coor)

gis_over_armoire_points = gpd.read_file(armoire_resa)
gis_over_coupe_liste_points = gpd.read_file(coupe_liste_resa)
gis_over_lg_rappel_lines = gpd.read_file(lg_rappel)
gis_over_noms_voiries_lines = gpd.read_file(noms_voiries)
gis_over_postes_points = gpd.read_file(postes_resa)
gis_over_supports_points = gpd.read_file(supports_resa)
gis_over_troncon_lines = gpd.read_file(troncon_resa)
gis_over_voirie_bord_lines = gpd.read_file(voirie_bord)

#Underground cables
gis_under_lines = gpd.read_file(under_lines)
gis_under_points = gpd.read_file(under_points)
gis_under_polys = gpd.read_file(under_polys)

df_sub_tfo = pd.read_excel(sub_tfo)
df_consumption = pd.read_excel(m105)
df_bi_assets = pd.read_excel(bi_assets)

city_of_interest = donotcommit.CityofInterest()


# gis_over_armoire_points['length'] = gis_over_armoire_points.length
gis_over_coupe_liste_points['length'] = gis_over_coupe_liste_points.length
gis_over_lg_rappel_lines['length'] = gis_over_lg_rappel_lines.length
gis_over_noms_voiries_lines['length'] = gis_over_noms_voiries_lines.length
# gis_over_postes_points['length'] = gis_over_postes_points.length
# gis_over_supports_points['length'] = gis_over_supports_points.length
gis_over_troncon_lines['length'] = gis_over_troncon_lines.length
gis_over_voirie_bord_lines['length'] = gis_over_voirie_bord_lines.length

gis_under_lines['length'] = gis_under_lines.length

In [ ]:
def ExploreDF(df, isgis):
    print('Sample DF:')
    print(df.head(2), '\n')
    print('Columns: ', list(df.columns), '\n')
    print(df.shape, '\n')
    null_values = df.isnull().sum()
    if(null_values.sum()>0):
        print(f'Null values: \n{null_values[null_values>0]}')
    if(isgis):
        return df.explore()

In [ ]:
import unidecode
def LocationTransformation(column):
	return column.apply(lambda x: unidecode.unidecode(str(x).lower()))

In [ ]:
print(gis_ean_coor.head(3))

#Check for null values
print(gis_ean_coor.shape)
print(gis_ean_coor.isnull().sum())
print()

gis_ean_coor.localiteOr = LocationTransformation(gis_ean_coor.localiteOr)
cities = set(gis_ean_coor.localiteOr)
print([i for i in cities if i.startswith(city_of_interest[:4])])
print()

#Check for null values for the city of interest
ver = gis_ean_coor[gis_ean_coor.localiteOr==city_of_interest]
print(ver.shape)
print(ver.isnull().sum().sum())

In [ ]:
#Simple plot 
ax = gis_under_points.plot()
ax = gis_under_lines.plot('length', ax=ax, legend=True)
gis_under_polys.plot('area', ax=ax, legend=True)

In [ ]:
ele = 7
if(ele==0):
    ax = ExploreDF(gis_over_armoire_points, True)
elif(ele==1):
    ax = ExploreDF(gis_over_coupe_liste_points, True)
elif(ele==2):
    ax = ExploreDF(gis_over_lg_rappel_lines, True)
elif(ele==3):
    ax = ExploreDF(gis_over_noms_voiries_lines, True)
elif(ele==4):
    ax = ExploreDF(gis_over_postes_points, True)
elif(ele==5):
    ax = ExploreDF(gis_over_supports_points, True)
elif(ele==6):
    ax = ExploreDF(gis_over_troncon_lines, True)
elif(ele==7):
    ax = ExploreDF(gis_over_voirie_bord_lines, True)
ax


In [ ]:
#Detailed plot (Belgium map)

#Overhead cables
legend = False
color = 'orangered'
ax = gis_over_armoire_points.explore(color='blue', legend=legend)
ax = gis_over_coupe_liste_points.explore(color='blue',m=ax, legend=legend)
ax = gis_over_lg_rappel_lines.explore(color=color, m=ax, legend=legend)
ax = gis_over_noms_voiries_lines.explore(color=color, m=ax, legend=legend)
ax = gis_over_postes_points.explore(color='green', m=ax, legend=legend)
ax = gis_over_supports_points.explore(color='blue', m=ax, legend=legend)
ax = gis_over_troncon_lines.explore(color=color, m=ax, legend=legend)
ax = gis_over_voirie_bord_lines.explore(color=color, m=ax, legend=not legend)

In [ ]:
#Points
ax1 = gis_over_coupe_liste_points.explore()
gis_over_armoire_points.explore(m=ax1, color='red')
gis_over_supports_points.explore(m=ax1, color='green')
gis_over_postes_points.explore(m=ax1, color='purple')

In [ ]:
# ax2 = gis_over_lg_rappel_lines.explore()
# gis_over_noms_voiries_lines.explore(m=ax2, color='red')
a = gis_over_troncon_lines.explore( color='green')
gis_over_voirie_bord_lines.explore(m=a, color='purple')
# ax2

In [ ]:
#Road Line + poles
ax = gis_over_supports_points.explore(color='purple')
gis_over_troncon_lines.explore(m=ax, color='green')

In [ ]:
#Detailed plot (Belgium map)
gis_under_lines['length'] = gis_under_lines.length
gis_under_polys['centroids'] = gis_under_polys.centroid
gis_under_polys['area'] = gis_under_polys.area

#Underground cables
color1 = 'purple'
ax = gis_under_points.explore(color=color1)
ax = gis_under_lines.explore(color='green', legend=True,m=ax)
gis_under_polys.explore(color='red', legend=True, m=ax)

Exploration xlsx files

In [ ]:
ExploreDF(df_sub_tfo, False)

In [ ]:
ExploreDF(df_consumption, False)

In [ ]:
ExploreDF(df_bi_assets, False)

In [ ]:
def CheckForCoI(df, col):
    df[col] = LocationTransformation(df[col])
    cities = set(df[col])
    print(sorted(cities))
    print(len(df[df[col] == city_of_interest]))
    print([i for i in cities if i.startswith(city_of_interest[:4])])

In [ ]:
print(df_sub_tfo.shape)
df_sub_tfo.head(3)

In [ ]:
col_ean = 'localiteOr'
CheckForCoI(gis_ean_coor, col_ean)

In [ ]:
col_com = 'Localité OR'
CheckForCoI(df_consumption, col_com)

In [ ]:
col_bi = 'LOCALITE_OR'
CheckForCoI(df_bi_assets, col_bi)

In [ ]:
#Filter for CoI
print(f'Before filtering: {gis_ean_coor.shape}')
gis_ean_coor_filtered = gis_ean_coor[gis_ean_coor[col_ean] == city_of_interest]
print(f'After filtering: {gis_ean_coor_filtered.shape}')
col_join = 'EAN'
gis_ean_coor_filtered.columns = [col_join, *list(gis_ean_coor_filtered.columns)[1:]]

from shapely.geometry import Point
gis_ean_coor_filtered['geometry'] = gis_ean_coor_filtered.apply(lambda x: Point(x.latWgs84, x.longWgs84), axis=1)

df_consumption_filtered = df_consumption[df_consumption[col_com] == city_of_interest]
df_bi_assets_filtered = df_bi_assets[df_bi_assets[col_bi] == city_of_interest]

In [ ]:
print(gis_ean_coor_filtered.shape)
print(df_consumption_filtered.shape)
print(df_bi_assets_filtered.shape)

In [ ]:
#Duplicate EAN
print(gis_ean_coor_filtered.EAN.shape)
print(len(set(gis_ean_coor_filtered.EAN)))

In [ ]:
gis_ean_coor_filtered[gis_ean_coor_filtered.duplicated('EAN',False)==True].sort_values('EAN')

In [ ]:
print(gis_ean_coor_filtered.loc[576464])
print(gis_ean_coor_filtered.loc[144086])

Joins

In [ ]:
#ASSUMPTION: the duplicated EAN are just dropped
gis_ean_coor_filtered = gis_ean_coor_filtered[gis_ean_coor_filtered.duplicated('EAN')==False]

gis_ean_coor_filtered = gis_ean_coor_filtered.loc[:,['EAN', 'latWgs84', 'longWgs84', 'geometry']]
print(gis_ean_coor_filtered.EAN.iloc[0], type(gis_ean_coor_filtered.EAN.iloc[0]))
gis_ean_coor_filtered['EAN'] = gis_ean_coor_filtered['EAN'].astype(float)
print(gis_ean_coor_filtered.EAN.iloc[0], type(gis_ean_coor_filtered.EAN.iloc[0]))
print(gis_ean_coor_filtered.shape)

In [ ]:
#https://epsg.io/transform
#https://pygis.io/docs/d_understand_crs_codes.html
# print(gis_over_armoire_points['geometry'].crs)
gis_over_armoire_points['geometry'].crs

In [ ]:
gis_ean_coor_filtered = gpd.GeoDataFrame(gis_ean_coor_filtered)
# gis_ean_coor_filtered = gis_ean_coor_filtered.set_crs(4326)
# gis_ean_coor_filtered['geometry'] = gis_ean_coor_filtered['geometry'].to_crs(31370)

In [ ]:
print(df_consumption_filtered.EAN.iloc[0], type(df_consumption_filtered.EAN.iloc[0]))
df_consumption_filtered = df_consumption_filtered.loc[:,['EAN', 'P Contractuelle Prél. (kVA)', 'P Production (kVA)']]
df_consumption_filtered['EAN'] = df_consumption_filtered['EAN'].astype(float)
print(df_consumption_filtered.EAN.iloc[0], type(df_consumption_filtered.EAN.iloc[0]))

In [ ]:
print(df_bi_assets_filtered.EAN.iloc[0], type(df_bi_assets_filtered.EAN.iloc[0]))
df_bi_assets_filtered = df_bi_assets_filtered.loc[:,['EAN', 'P_CONTRACTUELLE (kVA)', 'Numéro cabine']]
df_bi_assets_filtered['EAN'] = df_bi_assets_filtered['EAN'].astype(float)
print(df_bi_assets_filtered.EAN.iloc[0], type(df_bi_assets_filtered.EAN.iloc[0]))

In [ ]:
# GIS EAN with consumptions
df_cust_con = gis_ean_coor_filtered.merge(df_consumption_filtered, on='EAN', how='inner')
# gis_ean_coor_filtered['EAN'] = gis_ean_coor_filtered['ean'].astype(float)
# df_cust_con = df_consumption_filtered.merge( gis_ean_coor_filtered, on='EAN', how='inner')
print(gis_ean_coor_filtered[gis_ean_coor_filtered.EAN == 541456700000166555])
print(df_consumption_filtered[df_consumption_filtered.EAN == 541456700000166555])
print(df_cust_con.shape)

In [ ]:
# GIS EAN with bi_assets
df_cust_bi = gis_ean_coor_filtered.merge(df_bi_assets_filtered, on='EAN', how='inner')
print(gis_ean_coor_filtered[gis_ean_coor_filtered.EAN == 541456700000166555])
print(df_bi_assets_filtered[df_bi_assets_filtered.EAN == 541456700000166555])
print(df_cust_bi.shape)

Build Network

In [ ]:
gis_over_lg_rappel_lines.explore()

In [ ]:
import pandapower as pp

def GenerateBuses(net, df, s, color, name, old_buses):
	new_buses = old_buses
	vn_kv = 120
	colors = [] if 'color' not in net.bus.columns else list(net.bus['color'])
	sizes = [] if 'marker_size' not in net.bus.columns else list(net.bus['marker_size'])
	names = [] if 'name' not in net.bus.columns else list(net.bus['name'])
	counter = len(net.bus)
	for index, row in df.iterrows():
		geodata = list(row.geometry.coords)[0]
		if(geodata in new_buses.keys()):
			id = new_buses[geodata]
			print(f'There is a bus used more than once: GPS: {geodata}. Bus ID: {id}')
			# net.bus.loc[id,'color'] = 'black'
			# net.bus.loc[id,'size'] = s*2
			colors[id] = 'black'
			sizes[id] = s*2
		else:
			bus = pp.create_bus(net, name=f'Bus {counter+index}', vn_kv=vn_kv, geodata=geodata, type='b')
			colors.append(color)
			sizes.append(s)
			names.append(name)
			new_buses[geodata] = bus
	net.bus['color'] = colors
	net.bus['marker_size'] = sizes
	net.bus['name'] = names
	return new_buses

def GenerateLines(net, df, s, color, name, old_buses):
	type_bus = 'n'
	vn_kv = 120
	color_buses = [] if 'color' not in net.bus.columns else list(net.bus['color'])
	sizes = [] if 'marker_size' not in net.bus.columns else list(net.bus['marker_size'])
	names = [] if 'name' not in net.bus.columns else list(net.bus['name'])
	color_lines = [] if 'color' not in net.line.columns else list(net.line['color'])
	coor_lines = [] if 'coor' not in net.line.columns else list(net.line['coor'])
	type_lines = [] if 'type' not in net.line.columns else list(net.line['type'])
	new_buses = {} if old_buses is None else old_buses
	m = []
	counter_buses = len(net.bus)
	counter_lines = len(net.line)
	for index, row in df.iterrows():
		geodata = list(row.geometry.coords)

		from_gps = geodata[0]
		if(from_gps in new_buses.keys()):
			id = new_buses[from_gps]
			print(f'There is a (from) bus used more than once: GPS: {from_gps}. Bus ID: {id}')
			from_bus = id
			# colors.append(colors[id])
			# sizes.append(sizes[id])
			# m.append(from_bus)
		else:	
			from_bus = pp.create_bus(net, name=f'Bus (from) line {counter_buses + index}', type=type_bus, vn_kv=vn_kv, geodata=from_gps)
			# counter_buses+=1
			new_buses[from_gps] = from_bus
			color_buses.append(color)
			sizes.append(s)
			names.append(name)
		
		to_gps = geodata[-1]
		if(to_gps in new_buses.keys()):
			id = new_buses[to_gps]
			print(f'There is a (to) bus used more than once: GPS: {to_gps}. Bus ID: {id}')
			to_bus = id
			# to_bus = buses[to_gps]
			# colors.append('green')
			# sizes.append(10)
		else:	
			to_bus = pp.create_bus(net, name=f'Bus (to) line {counter_buses + index}', vn_kv=vn_kv, geodata=to_gps)
			# counter_buses+=1
			new_buses[to_gps] = to_bus
			color_buses.append(color)
			sizes.append(s)
			names.append(name)

		length = row.length
		line = pp.create_line(net, from_bus=from_bus, to_bus=to_bus, std_type='NAYY 4x50 SE', length_km=length/1000.0, name=f'{name} {counter_lines + index}', geodata=geodata)
		color_lines.append(color)
		coor_lines.append(geodata)
		type_lines.append(name)
	net.bus['color'] = color_buses
	net.bus['marker_size'] = sizes
	net.bus['name'] = names
	net.line['color'] = color_lines
	net.line['coor'] = coor_lines
	net.line['type'] = type_lines
	return new_buses, m


In [ ]:
#CLEANING
#Duplicated GPS location in gis_over_coupe_liste_points
#ASSUMPTION: The duplicated are simply dropped 
duplicated = gis_over_coupe_liste_points[gis_over_coupe_liste_points.duplicated('geometry',False)==True]

# print(duplicated)
print(len(duplicated)/2)

#Remove duplicated
print(gis_over_coupe_liste_points.shape)
gis_over_coupe_liste_points = gis_over_coupe_liste_points[gis_over_coupe_liste_points.duplicated('geometry')==False]
print(gis_over_coupe_liste_points.shape)

In [ ]:
gis_over_coupe_liste_points[gis_over_coupe_liste_points.duplicated('geometry',False)==True]

In [ ]:
# gis_ean_coor['geometry'] = gis_ean_coor.apply(lambda x: np.array(x.latWgs84, x.longWgs84), axis=1)
# gis_ean_coor[gis_ean_coor.duplicated('geometry',False)==True]
gis_ean_coor_filtered[gis_ean_coor_filtered.duplicated('geometry',False)==True]

In [ ]:
gis_ean_coor_filtered[gis_ean_coor_filtered.geometry == a]

In [ ]:
import plotly.graph_objects as go
    # gis_over_armoire_points
    # gis_over_coupe_liste_points
# gis_over_lg_rappel_lines
# gis_over_noms_voiries_lines
    # gis_over_postes_points
    # gis_over_supports_points
# gis_over_troncon_lines
# gis_over_voirie_bord_lines

net = pp.create_empty_network()

s = 6
buses = {}
# types = ['armoire', 'coupe', 'postes (poles)', 'supports', 'underground', 'customers']
points_types = ['armoire', 'support (poles)', 'coupe', 'postes', 'underground']

# Overhead
buses = GenerateBuses(net, gis_over_armoire_points, s, 'red', points_types[0], buses)

buses = GenerateBuses(net, gis_over_supports_points, s, 'green', points_types[1], buses)

buses = GenerateBuses(net, gis_over_coupe_liste_points, s, 'blue', points_types[2], buses)

buses = GenerateBuses(net, gis_over_postes_points, s, 'purple', points_types[3], buses)


# Undedrground
buses = GenerateBuses(net, gis_under_points, s, 'black', points_types[4], buses)

# Customers
# buses = GenerateBuses(net, df_cust_bi, s, 'lime', 'customers', buses)

In [ ]:
fig = go.Figure()
for t in points_types:
    fig.add_trace(go.Scatter(x=net.bus_geodata[net.bus.name==t]['x'],
                            y=net.bus_geodata[net.bus.name==t]['y'],
                            mode='markers', marker=dict(size=net.bus.marker_size), name=t, showlegend=True))
fig.update_layout(
        autosize=False,
        width=1000,
        height=600, 
        margin=dict(
            l=50,
            r=50,
            b=50,
            t=50,
            pad=4)
        )
# fig.update_yaxes(automargin=True)
fig.show()

In [ ]:
# import time

# gis_over_armoire_points
# gis_over_coupe_liste_points
    # gis_over_lg_rappel_lines
    # gis_over_noms_voiries_lines
# gis_over_postes_points
# gis_over_supports_points
    # gis_over_troncon_lines
    # gis_over_voirie_bord_lines

# net = pp.create_empty_network()

# t1 = time.time()
s = 6
# buses = None
#Overhead
buses, m = GenerateLines(net, gis_over_lg_rappel_lines, s, 'lime', 'lg_rappel_line', buses)
print()
# buses, m = GenerateLines(gis_over_noms_voiries_lines, s, 'lime', buses) #There are multi-line objects here
print()
# buses, m = GenerateLines(gis_over_troncon_lines, s, 'lime', buses) #There are multi-line objects here
print()
# buses, m = GenerateLines(gis_over_voirie_bord_lines, s, 'green', buses) #There are multi-line objects here
print()

#Underground
# buses, m = GenerateLines(gis_under_lines, buses)
# t2 = time.time()
# print(f'Execution time: {(t2-t1):.3f}s')

In [ ]:
fig = go.Figure()
for t in points_types:
    fig.add_trace(go.Scatter(x=net.bus_geodata[net.bus.name==t]['x'],
                            y=net.bus_geodata[net.bus.name==t]['y'],
                            mode='markers', marker=dict(size=net.bus.marker_size), name=t, showlegend=True))

for l in net.line.iterrows():
    l = l[1]
    xs = [i[0] for i in l.coor]
    ys = [i[1] for i in l.coor]
    fig.add_trace(go.Scatter(x=xs,
                            y=ys,
                            mode='lines', line=dict(color=l.color), name=l.name, showlegend=False))
# fig.add_trace(go.Scatter(x=[ net.bus_geodata.iloc[l.from_bus]['x'], net.bus_geodata.iloc[l.to_bus]['x'] ] ,
#                         y=[ net.bus_geodata.iloc[l.from_bus]['y'], net.bus_geodata.iloc[l.to_bus]['y'] ],
#                         mode='lines+markers', line=dict(color=l.color), name=l['name'].split(' ')[:-1], showlegend=True))

fig.update_layout(
        autosize=False,
        width=1000,
        height=600, 
        margin=dict(
            l=50,
            r=50,
            b=50,
            t=50,
            pad=4)
        )

# fig.update_yaxes(automargin=True)
fig.show()

In [ ]:
import pandapower.plotting as plot
traces = []
for t in points_types:
    print(t)
    buses = net.bus[net.bus.name==t]
    trace_name = buses.name.head(1).to_list()[0]
    trace = pp.plotting.plotly.create_bus_trace(net, buses=buses.index, size=buses.marker_size, color=buses.color, trace_name=trace_name)
    traces.append(trace[0])

lines = net.line[net.line.type=='lg_rappel_line']
trace_name = lines.type.head(1).to_list()[0]
trace_color = lines.color.head(1).to_list()[0]
info = lines.name
trace = pp.plotting.plotly.create_line_trace(net, lines=lines.index, width=3, color=trace_color, trace_name=trace_name, infofunc=info)
[traces.append(t) for t in trace]


plot.draw_traces(traces)
plt.show()

In [ ]:
for i,j in gis_over_noms_voiries_lines.iterrows():
    print(i,j)
    jj = list(j.geometry.coords)
    print(jj)
    # break

In [ ]:
plot = simple_plotly(net, bus_size=net.bus['size'], bus_color=net.bus['color'])

In [ ]:
#It is okay. That's how life is! D:
# gis_over_noms_voiries_lines, gis_over_troncon_lines, gis_over_voirie_bord_lines
df = gis_over_voirie_bord_lines
ind = []
jj = set()
for i in df.iterrows():
    jj.add(type(i[1].geometry))
    if(str(type(i[1].geometry))=="<class 'shapely.geometry.multilinestring.MultiLineString'>"):
        print(str(type(i[1].geometry)))
        ind.append(i[1].name)
    # break

In [ ]:
jj

In [ ]:
ind[:10]

In [ ]:
df.loc[ind].explore()

In [ ]:
i = ind[-10]
print(len(df.loc[i].geometry))
df.loc[i].geometry

In [ ]:
list(df.iloc[ind[0]].geometry.geoms)